In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preprocessing

In [ ]:
fake = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")
true = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")

In [ ]:
fake.head()

In [ ]:
fake['prediction'] = 0

In [ ]:
true['prediction'] = 1

In [ ]:
fake.drop(labels=['subject','date'],axis=1,inplace=True)
true.drop(labels=['subject','date'],axis=1,inplace=True)

In [ ]:
data = pd.concat([fake,true])

In [ ]:
data = data.sample(len(data))

In [ ]:
data.head()

In [ ]:
data['text_'] = data['title']+" "+data['text']

In [ ]:
data.head()

In [ ]:
data = data[['text_','prediction']]

In [ ]:
data.head()

# Splitting the Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.text_, data.prediction, test_size=0.1, random_state=37)

# Tokenizing and Padding Text

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tk = Tokenizer(num_words=1000,
filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n',lower=True, split=" ")
tk.fit_on_texts(X_train)
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=100)
X_test_seq_trunc = pad_sequences(X_test_seq, maxlen=100)

# Model (CNN)

In [ ]:
from tensorflow.keras import models,layers
emb_model = models.Sequential()
emb_model.add(layers.Embedding(tk.num_words, 8, input_length=100))
emb_model.add(layers.Convolution1D(16,4,activation='relu'))
emb_model.add(layers.AveragePooling1D())
emb_model.add(layers.Convolution1D(32,4,activation='relu'))
emb_model.add(layers.AveragePooling1D())
emb_model.add(layers.Flatten())
emb_model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
import tensorflow as tf
emb_model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
history = emb_model.fit(x=X_train_seq_trunc,y=y_train,batch_size=256,epochs=100,validation_data=(X_test_seq_trunc,y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])

In [ ]:
plt.plot(history.history['val_loss'])

## You can see that the validation loss is increasing after 12 epochs after 12 epochs the model is getting overfitted

 ### Retraining the model for 12 epochs

In [ ]:
history = emb_model.fit(x=X_train_seq_trunc,y=y_train,batch_size=256,epochs=12,validation_data=(X_test_seq_trunc,y_test))